In [ ]:
!pip install mlrun

### Change this to the address of your Iguazio Docker registry 
Get the address from the Services list (Docker Registry)

In [1]:
docker_registry = 'docker-registry.default-tenant.app.mdl0521.iguazio-cd0.com:80'

### Import MLRun functions

In [2]:
from mlrun import mlconf, code_to_function, mount_v3io, NewTask

### Set general paramters to send to all runs
When you upload files to our platform to process you can refer to the same location in Jupyter, shell and your jobs

In [3]:
general_params={"param1" : "one", "param2" : 2, 'file_location': "/User/data"}

### Create MLRun execution artifacts
* code_to_function takes your python code, base64 encodes it into the function object
    * note the paramter filename. That is your python code location
* In this line we are also telling MLRun that the job will be a Kubernetes job
* The mount_v3io ( without parameters ), when runnign in-platform mount you home directory in /User same as in Jupyter and shell

In [4]:
# create the function from the notebook code + annotations, add volumes
fn = code_to_function('training_job', kind='job',filename='/User/igzcustomers/ucsd/script_to_run.py').apply(mount_v3io())

### Function specs
* After your image is in our repo you can use it here
* Change the numbers of GPUs you want allocated per run
    * Note: Oversubscribing will cause the job to wait indefinetly

In [45]:
fn.spec.image = docker_registry + '/iguazio/tfmodels'
#fn.spec.image_pull_policy = 'Always'
fn.spec.resources['limits']={'nvidia.com/gpu' : 1}

In [46]:
# If you want to see what the functions yaml looks like and also usefull for troubleshooting
#print(fn.to_yaml())

In [47]:
# You migth not save any artifacts from your code today, but it is required for MLRun
artifact_path="/User/artifact"

### Trigger the execution of multiple instances of the function

In [48]:
# This parameter is different for every run
run_params = ["one","two","three"]

* Loop through run_params and trigger a new instance of the function execution
* I append the "run ID" as a paramter called run
* watch=False delegates the execution to MLRun and returns immediately
* I give each run a different name to watch each in the MLRun UI separately
* After you run this, go to the MLRun UI to view the jobs
    * They will be under Project ucsd-igz

In [49]:
for each in run_params:
    general_params['run'] = each
    task = NewTask(project='ucsd-igz',name='parallel_example_%s'% each, params=general_params, artifact_path=artifact_path)
    myrun = fn.run(task,handler='handler',watch=False)

[mlrun] 2020-06-30 13:35:54,932 starting run parallel_example_one uid=bc0b1876fce2487abf4bb5b8f9ba5672  -> http://mlrun-api:8080
[mlrun] 2020-06-30 13:35:55,066 Job is running in the background, pod: parallel-example-one-8mvf6


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
ucsd-igz,...f9ba5672,0,Jun 30 13:35:55,running,parallel_example_one,v3io_user=adminkind=jobowner=admin,,param1=oneparam2=2file_location=/User/datarun=one,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run bc0b1876fce2487abf4bb5b8f9ba5672 --project ucsd-igz , !mlrun logs bc0b1876fce2487abf4bb5b8f9ba5672 --project ucsd-igz
[mlrun] 2020-06-30 13:35:55,083 run executed, status=running
[mlrun] 2020-06-30 13:35:55,084 starting run parallel_example_two uid=a33814a49f6f4be3b37aed9380757274  -> http://mlrun-api:8080
[mlrun] 2020-06-30 13:35:55,212 Job is running in the background, pod: parallel-example-two-7rz4z


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
ucsd-igz,...80757274,0,Jun 30 13:35:55,running,parallel_example_two,v3io_user=adminkind=jobowner=admin,,param1=oneparam2=2file_location=/User/datarun=two,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run a33814a49f6f4be3b37aed9380757274 --project ucsd-igz , !mlrun logs a33814a49f6f4be3b37aed9380757274 --project ucsd-igz
[mlrun] 2020-06-30 13:35:55,232 run executed, status=running
[mlrun] 2020-06-30 13:35:55,234 starting run parallel_example_three uid=41d5ab0ece5e4c419a68c2d96ff0daab  -> http://mlrun-api:8080
[mlrun] 2020-06-30 13:35:55,378 Job is running in the background, pod: parallel-example-three-4fhzq


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
ucsd-igz,...6ff0daab,0,Jun 30 13:35:55,running,parallel_example_three,v3io_user=adminkind=jobowner=admin,,param1=oneparam2=2file_location=/User/datarun=three,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 41d5ab0ece5e4c419a68c2d96ff0daab --project ucsd-igz , !mlrun logs 41d5ab0ece5e4c419a68c2d96ff0daab --project ucsd-igz
[mlrun] 2020-06-30 13:35:55,396 run executed, status=running
